In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
import time
import urllib.parse
from dotenv import load_dotenv
import os
from requests.exceptions import HTTPError, Timeout, RequestException
from ratelimit import limits, sleep_and_retry

# Load environment variables
load_dotenv()
api = os.getenv('api')
db_password = os.getenv('DB_PASSWORD')

# Database connection parameters
server = '35.198.20.9'  # Replace with your GCP SQL Server public IP
port = 1433
database = 'RiotDB'     # Replace with your database name
username = 'sqlserver'  # Replace with your username

# Construct the database connection string
connection_url = (
    f"mssql+pyodbc://{username}:{db_password}@{server}:{port}/{database}"
    "?driver=ODBC+Driver+18+for+SQL+Server"
    "&Encrypt=yes"
    "&TrustServerCertificate=yes"
)

# Database engine
engine = create_engine(connection_url)

# Rate limiting (100 requests per 2 minutes)
CALLS = 100
TIME_PERIOD = 120  # 120 seconds

# Decorator to apply rate limiting
@sleep_and_retry
@limits(calls=CALLS, period=TIME_PERIOD)
def call_api(url):
    """Calls the Riot API with the given URL and handles errors."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes (4xx, 5xx)
        return response.json()
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except RequestException as req_err:
        print(f"Other request error occurred: {req_err}")
    return None

def get_puuid(nome_invocador, tag_invocador):
    """Fetches the PUUID of the summoner based on their summoner name and tag."""
    encoded_name = urllib.parse.quote(nome_invocador)
    api_invocador = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{encoded_name}/{tag_invocador}?api_key={api}"
    response = call_api(api_invocador)
    return response.get('puuid') if response else None

def get_match_ids(puuid_invocador):
    """Retrieves the last 100 match IDs for a given PUUID."""
    api_partidas = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid_invocador}/ids?start=0&count=100&api_key={api}"
    return call_api(api_partidas)

def fetch_match_details(match_id):
    """Fetches match details for a specific match ID."""
    api_match_details = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api}"
    return call_api(api_match_details)

def main():
    # Extract the summoner's PUUID
    nome_invocador = input("Escreva o nome do invocador: ")
    tag_invocador = input("Digite a tag do invocador: ")

    print("Retrieving PUUID...")
    with tqdm(total=1, desc="Fetching PUUID") as pbar:
        puuid_invocador = get_puuid(nome_invocador, tag_invocador)
        pbar.update(1)

    if puuid_invocador:
        print("PUUID retrieved successfully.")
        
        # Extract the last 100 match IDs
        print("Retrieving match IDs...")
        match_ids = []
        with tqdm(total=1, desc="Fetching match IDs") as pbar:
            match_ids = get_match_ids(puuid_invocador)
            pbar.update(1)

        if match_ids:
            print("Saving match IDs to database...")
            with tqdm(total=1, desc="Saving match IDs") as pbar:
                df_match_id = pd.DataFrame(match_ids, columns=['Match_ID'])
                df_match_id.to_sql('MatchID_Invocador', con=engine, if_exists='replace', index=False)
                pbar.update(1)
            print("Match IDs saved to database.")
        else:
            print("No match IDs retrieved.")
            return

        # Retrieve match details for the match IDs and filter for the last 20 ranked games (queueId == 420)
        print("Retrieving match details for the last 20 ranked games...")
        details_match = []
        ranked_games_collected = 0  # Counter to stop at 20 ranked games

        for match_id in tqdm(match_ids, desc="Processing match IDs"):
            if ranked_games_collected >= 20:
                break  # Stop once we have 20 ranked games

            match_detail = fetch_match_details(match_id)
            if match_detail:
                queue_id = match_detail.get('info', {}).get('queueId')
                participants = match_detail.get('info', {}).get('participants', [])
                participants_df = pd.json_normalize(participants)
                participants_df.insert(0, 'Match_ID', match_id)

                if queue_id == 420:  # Only ranked solo queue
                    details_match.append(participants_df)
                    ranked_games_collected += 1  # Increment the counter

            time.sleep(0.5)  # Respect API request limit

        # Concatenate and save match details to the database
        print("Processing and saving match details...")
        if details_match:
            with tqdm(total=1, desc="Saving match details to database") as pbar:
                df_matchdetails = pd.concat(details_match, ignore_index=True)
                df_matchdetails.to_sql('Detalhes_Liga_Invocador', con=engine, if_exists='replace', index=False)
                pbar.update(1)
            print("Match details saved successfully.")
        else:
            print("No valid match details found.")

    else:
        print("Failed to retrieve PUUID for the summoner.")

if __name__ == "__main__":
    main()


Retrieving PUUID...


Fetching PUUID: 100%|██████████| 1/1 [00:01<00:00,  1.42s/it]


PUUID retrieved successfully.
Retrieving match IDs...


Fetching match IDs: 100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


Saving match IDs to database...


...

Match IDs saved to database.
Retrieving match details for the last 20 ranked games...


Processing match IDs:  34%|███▍      | 20/59 [00:32<01:03,  1.63s/it]


Processing and saving match details...


Saving match details to database: 100%|██████████| 1/1 [00:06<00:00,  6.72s/it]

Match details saved successfully.
